# Exemplos de queries SQL integrados ao Python e Pandas
Usaremos dados de exemplo do site [MySQL Tutorial](http://www.mysqltutorial.org/mysql-sample-database.aspx).

Para recriar o banco de dados em SQLite, sem Python, basta executar o arquivo com os comandos SQL na linha de comando do sistema operacional:

```shell
sqlite3 database.db < sample-database-dump.sql
```

O arquivo `database.db` será criado e tabelas e dados conforme especificados em `sql-sample-database.sql` serão populados.

Se estivéssemos lidando com um SGBD (sistema gerenciador de banco de dados) mais robusto, como MariaDB/MySQL, Oracle ou DB2, o comando acima seria diferente e deverá conter:

* o hostname ou IP do servidor de banco de dados
* usuário e senha de acesso
* nome do banco de dados em que vamos operar

Mas SQLite é bem mais simples e didático e opera num arquivo local.

## Visualizando o banco de dados, suas tabelas e dados

<img src="MySQL-Sample-Database-Schema.png">

Cada SGBD tem seu próprio ferramental para navegar nos dados. MariaDB/MySQL tem o popular [PHP MyAdmin](https://www.phpmyadmin.net), Oracle e DB2 tem suas próprias ferramentas proprietárias, SQLite tem [DB Browser for SQLite](https://sqlitebrowser.org) e uma ferramenta online chamada [SQLite Online](https://sqliteonline.com).

Para navegar nos dados do DB SQLite que acabamos de criar, use uma das 3 opções:

* Use o comando `sqlite3`.
* Instale o **DB Browser for SQLite** e abra o arquivo.
* Envie o arquivo para o site **SQLiteOnline.com**.

## SQLite

#### Vamos primeiro usar a estrutura e dados do arquivo `sql-sample-database.sql` para criar um banco de dados no arquivo `database.db`.

#### Usamos o método [`connect()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection) para criar um objeto de conexão com esse banco de dados.

In [5]:
import pandas as pd
import sqlite3

db = sqlite3.connect('database.db')

#### Depois de ter um objeto de conexão, podemos criar um [`objeto Cursor`](https://docs.python.org/2.5/lib/sqlite3-Cursor-Objects.html) [`cursor()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor) e chamar seu método [`execute()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute) para executar comandos `SQL`.

In [6]:
script = 'sample-database-dump.sql'

db.cursor().executescript(open(script).read())

#### Camos criar uma conexão com a nossa base de dados.

In [7]:
conn = sqlite3.connect('database.db')
curs = conn.cursor()
#conn.close()

In [8]:
query = """CREATE TABLE IF NOT EXISTS countries (

key INTEGER PRIMARY KEY AUTOINCREMENT,

name text UNIQUE,

founding_year INTEGER,

capital TEXT

);"""

#query = """DROP TABLE countries ;"""

curs.execute(query)

conn.commit()

#### Vamos criar agora uma tabela.

In [9]:
#query = "DELETE FROM countries WHERE name == 'BRASIL';"

query  = """INSERT INTO countries (key, name, founding_year, capital)
    VALUES (1, 'BRASIL', 1500, 'SALVADOR')
;"""

conn.commit()

curs.execute(query)

#### Vamos observar a tabela que foi criada.

In [10]:
query = """SELECT * 
    FROM countries
;"""

queryResult = pd.read_sql_query(query, db)

queryResult.head()

,key,name,founding_year,capital


In [11]:
res = curs.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
for name in res:
    print (name[0])

my_table
sqlite_sequence
countries
customers
employees
offices
orderdetails
orders
payments
productlines
products


#### Vamos inserir valores na tabela criada.

In [12]:
#curs.execute("INSERT INTO countries (key, name, founding_year, capital) VALUES (1, 'BRASIL', 1500, 'SALVADOR');")
#conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query, db)

queryResult.head()

,key,name,founding_year,capital


#### Vamos inserir mais alguns países.

In [13]:
curs.execute("""INSERT INTO countries (key, name, founding_year, capital)

VALUES 
(2, 'MÉXICO', 1519, 'CIDADE DO MÉXICO'),
(3, 'ARGENTINA', 1516, 'BUENOS AIRES')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,SALVADOR
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES


#### Vamos inserir o país "EUA".

In [14]:
curs.execute("INSERT INTO countries (key, name, founding_year, capital) VALUES (4, 'EUA', 1585, 'St. Augustine');")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,SALVADOR
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES
3,4,EUA,1585,St. Augustine


#### Inserimos também o país "VENEZUELA".

In [15]:
curs.execute("""INSERT INTO countries  

(name, founding_year, capital)

VALUES 

('VENEZUELA', 1519, 'CARACAS')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,SALVADOR
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES
3,4,EUA,1585,St. Augustine
4,5,VENEZUELA,1519,CARACAS


#### Podemos também fazer uma atualização de valores na tabela criada.

In [16]:
curs.execute("""UPDATE countries  
SET capital = 'BRASILIA'

WHERE
name = 'BRASIL' AND founding_year = 1500

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,BUENOS AIRES
3,4,EUA,1585,St. Augustine
4,5,VENEZUELA,1519,CARACAS


#### É possível ainda excluir valores.

In [17]:

curs.execute("""UPDATE countries  

SET capital = NULL

WHERE

name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,3,ARGENTINA,1516,None
3,4,EUA,1585,St. Augustine
4,5,VENEZUELA,1519,CARACAS


#### Ou também excluir uma linha inteira.

In [18]:
curs.execute("""DELETE FROM countries  

WHERE
name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,key,name,founding_year,capital
0,1,BRASIL,1500,BRASILIA
1,2,MÉXICO,1519,CIDADE DO MÉXICO
2,4,EUA,1585,St. Augustine
3,5,VENEZUELA,1519,CARACAS


#### Para excluir uma tabela, usamos o comando [`DROP TABLE`](https://www.sqlitetutorial.net/sqlite-drop-table/).

<img src="RobertDROPTABLEStudants.png">

In [19]:
curs.execute("""
DROP TABLE countries;""")

conn.commit()

In [20]:
res = conn.execute("""SELECT name 
FROM sqlite_master 
WHERE type = 'table'
;"""
                  )
for name in res:
    print (name[0])

my_table
sqlite_sequence
customers
employees
offices
orderdetails
orders
payments
productlines
products


## Vamos focar em ler dados de um banco relacional utilizando SQL

#### A operação mais comum em um banco de dados é a leitura de dados e para isso precisamos escrever uma requisição, também conhecida como `QUERY`.

Uma `query` é construída em etapas:

1. [`SELECT`](https://www.sqlitetutorial.net/sqlite-select/) = irá iniciar a seleção de dados que faremos a leitura

2. FROM = irá selecionar as tabelas de interesse

3. [`WHERE`](https://www.sqlitetutorial.net/sqlite-where/) = vamos declarar as condições para ler os dados

4. [`GROUP BY`](https://www.sqlitetutorial.net/sqlite-group-by/) = podemos agrupar algum dado considerando uma coluna 

5. Pós processamento = podemos ordenar ([`ORDER BY`](https://www.sqlitetutorial.net/sqlite-order-by/)) ou estabelecer limites ([`LIMIT`](https://www.sqlitetutorial.net/sqlite-limit/))



## Vamos construir aos poucos nossa query

In [21]:
query = "SELECT * FROM customers;"

queryResult = pd.read_sql_query(query, db)

queryResult.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700


#### Vamos ler apenas algumas colunas específicas.

In [22]:
query = "SELECT customerName, phone FROM customers;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,customerName,phone
0,Atelier graphique,40.32.2555
1,Signal Gift Stores,7025551838
2,"Australian Collectors, Co.",03 9520 4555
3,La Rochelle Gifts,40.67.8555
4,Baane Mini Imports,07-98 9555


#### Vamos utilizar pela primeira vez uma condição para fazer a leitura.

In [23]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country = 'USA' ;"""

query2 = pd.read_sql_query(query,db)

query2.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800
1,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,None,San Rafael,CA,97562,USA,1165,210500
2,129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,None,San Francisco,CA,94217,USA,1165,64600
3,131,Land of Toys Inc.,Lee,Kwai,2125557818,897 Long Airport Avenue,None,NYC,NY,10022,USA,1323,114900
4,151,Muscle Machine Inc,Young,Jeff,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,1286,138500


#### Agora uma condição composta.

In [24]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country = 'USA' OR country = 'France';"""

query2 = pd.read_sql_query(query,db)

#query2.shape
query2.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800
2,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370,118200
3,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,None,San Rafael,CA,97562,USA,1165,210500
4,129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,None,San Francisco,CA,94217,USA,1165,64600


#### Podemos usar o comando [`NOT IN`](https://www.ramosdainformatica.com.br/banco_de_dados/sql-not-in-ou-not-exists/)

In [25]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country NOT IN ('USA', 'France');"""

query2 = pd.read_sql_query(query,db)

#query2.shape
query2.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
1,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700
2,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,None,Warszawa,None,01-012,Poland,NaN,0
3,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,None,Frankfurt,None,60528,Germany,1504.0,59700
4,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",None,Madrid,None,28034,Spain,1370.0,227600


#### Uma terceira seleção composta.

In [26]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country = 'USA' AND city = 'NYC';"""

query2 = pd.read_sql_query(query,db)

query2.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,131,Land of Toys Inc.,Lee,Kwai,2125557818,897 Long Airport Avenue,None,NYC,NY,10022,USA,1323,114900
1,151,Muscle Machine Inc,Young,Jeff,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,1286,138500
2,181,Vitachrome Inc.,Frick,Michael,2125551500,2678 Kingston Rd.,Suite 101,NYC,NY,10022,USA,1286,76400
3,424,Classic Legends Inc.,Hernandez,Maria,2125558493,5905 Pompton St.,Suite 750,NYC,NY,10022,USA,1286,67500
4,456,Microscale Inc.,Choi,Yu,2125551957,5290 North Pendale Street,Suite 200,NYC,NY,10022,USA,1286,39800


#### Para o caso de termos mais de uma tabela e quisermos ler apenas algumas colunas de cada tabela, utilzamos o [`JOIN`](https://www.sqlitetutorial.net/sqlite-join/).

In [27]:
query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM 
    orderdetails AS o INNER JOIN products AS p
/*
    orderdetails AS o,products AS p    
*/
    
    WHERE
    o.productCode=p.productCode;"""

queryResult = pd.read_sql_query(query,db)

#queryResult.shape
queryResult.head()

,priceEach,productCode,productName
0,136.00,S18_1749,1917 Grand Touring Sedan
1,55.09,S18_2248,1911 Ford Town Car
2,75.46,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport
3,35.29,S24_3969,1936 Mercedes Benz 500k Roadster
4,108.06,S18_2325,1932 Model A Ford J-Coupe


#### Para ordenar as linhas pelos valores de uma coluna em ordem decrescente devemos utilizar o comando [`DESC`](https://www.tutorialspoint.com/sqlite/sqlite_order_by.htm) após a coluna de referência.

In [28]:

query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode = p.productCode
    
    ORDER BY
    o.priceEach DESC;"""

queryResult = pd.read_sql_query(query,db)

#queryResult.shape
queryResult.head()

,priceEach,productCode,productName
0,214.30,S10_1949,1952 Alpine Renault 1300
1,214.30,S10_1949,1952 Alpine Renault 1300
2,214.30,S10_1949,1952 Alpine Renault 1300
3,212.16,S10_1949,1952 Alpine Renault 1300
4,210.01,S10_1949,1952 Alpine Renault 1300


#### O comando [`GROUP BY`](https://www.sqlitetutorial.net/sqlite-group-by/) é utilizado para agrupar um conjunto de linhas em acordo com um determinado atributo comum. Se usado em conjunto com o[`Order by`](https://www.sqlitetutorial.net/sqlite-order-by/), deve-se lembrar que order by segue group by em [ordem de operação](https://learnsql.com/blog/sql-order-of-operations/#:~:text=Six%20Operations%20to%20Order%3A%20SELECT,developer%20to%20know%20this%20order.).

In [29]:
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode = p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,Vendedor,VendaTotal
0,Studio M Art Models,474426.49
1,Red Start Diecast,655487.61
2,Highway 66 Mini Classics,664508.39
3,Carousel DieCast Legends,667190.00
4,Min Lin Diecast,680657.99


#### Podemos realizar a contagem de valores com o comando  [`COUNT`](https://www.sqlitetutorial.net/sqlite-count-function/).

In [30]:
query = """SELECT
    p.productVendor AS Vendedor,
    COUNT(p.productVendor) AS 'TotalRegistro'
    
    FROM 
    products AS p
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    TotalRegistro;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,Vendedor,TotalRegistro
0,Red Start Diecast,7
1,Autoart Studio Design,8
2,Min Lin Diecast,8
3,Second Gear Diecast,8
4,Studio M Art Models,8


## Lista completa de agregações 

<img src="list_aggregation.png">

#### O comando [`HAVING`](https://www.sqlitetutorial.net/sqlite-having/) é usado para especificar uma condição de pesquisa para um grupo.

In [31]:

query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o INNER JOIN products AS p
    
    WHERE
    o.productCode = p.productCode
    
    GROUP BY
    p.productVendor
    
    HAVING
    o.quantityOrdered < 30
    
    ORDER BY
    VendaTotal DESC
    ;"""

queryResult = pd.read_sql_query(query,db)

queryResult

,Vendedor,VendaTotal
0,Classic Metal Creations,934554.42
1,Unimax Art Galleries,884167.33
2,Gearbox Collectibles,828013.76
3,Highway 66 Mini Classics,664508.39
4,Studio M Art Models,474426.49


#### Podemos limitar a quantidade de linhas que serão lidas com o commando [`LIMIT`](https://www.sqlitetutorial.net/sqlite-limit/).

In [32]:
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal DESC
    
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query,db)

queryResult

,Vendedor,VendaTotal
0,Classic Metal Creations,934554.42
1,Unimax Art Galleries,884167.33
2,Gearbox Collectibles,828013.76


## SQLite com Pandas

###  <span style = "color:blue">Prática independente.</span>

#### Agora começamos a fazer queries:

## Lista de funcionários por chefe.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   employees e1,
   employees e2

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   e1.reportsTo = e2.employeeNumber

MOSTRE AS COLUNAS
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee

ORDENE POR
   boss
```

In [36]:
query = """SELECT
    (e2.firstName||' '||e2.lastName) AS boss,
    e2.jobTitle AS department,
    e1.employeeNumber AS ID,
    (e1.firstName||' '||e1.lastName) AS employee
    
    FROM 
    employees AS e1 INNER JOIN employees AS e2
    
    WHERE
    e1.reportsTo = e2.employeeNumber
    
    ORDER BY
    boss;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

,boss,department,ID,employee
0,Anthony Bow,Sales Manager (NA),1165,Leslie Jennings
1,Anthony Bow,Sales Manager (NA),1166,Leslie Thompson
2,Anthony Bow,Sales Manager (NA),1188,Julie Firrelli
3,Anthony Bow,Sales Manager (NA),1216,Steve Patterson
4,Anthony Bow,Sales Manager (NA),1286,Foon Yue Tseng


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee    

    FROM 
    employees AS e1, employees AS e2

    ORDER BY boss 
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult
-->

#### Inspeção de pedidos de compra.

#### Dados sobre pedidos aparecem nas tabelas `orders` e `orderdetails` e o que relaciona elas é a coluna `orderNumber`. Vamos inspecionar um único pedido.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   orderNumber=10103

MOSTRE AS COLUNAS
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
ORDENE POR
   orderLineNumber
```

In [37]:
query = """ SELECT orderLineNumber,
  productCode,
  priceEach,
  quantityOrdered,
  priceEach * quantityOrdered AS priceTotal
  
  FROM 
  orderdetails
  
  WHERE 
  orderNumber = 10103
  
  ORDER BY
  orderLineNumber;
  """

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,orderLineNumber,productCode,priceEach,quantityOrdered,priceTotal
0,1,S24_2300,107.34,36,3864.24
1,2,S18_2432,58.34,22,1283.48
2,3,S32_1268,92.46,31,2866.26
3,4,S10_4962,119.67,42,5026.14
4,5,S18_4600,98.07,36,3530.52


#  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal,
    orderNumber

    FROM 
    orderdetails

    WHERE orderNumber = 10103
    
    ORDER BY orderLineNumber
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Vamos melhorar a leitura dessa tabela adicionando a descrição do produto. Faremos um `JOIN` com a tabela `products`.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails AS o,
   products AS p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode AND
    orderNumber=10103

MOSTRE AS COLUNAS
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
ORDENE POR
   orderLineNumber
```

In [38]:
query = """SELECT 
        o.orderLineNumber,
        o.priceEach,
        o.quantityOrdered,
        o.priceEach * o.quantityOrdered AS itemTotal,
        o.productCode,
        p.productName

    FROM
    orderdetails AS o INNER JOIN products AS p

    WHERE
    o.productCode = p.productCode AND orderNumber = 10103    

    ORDER BY
    orderLineNumber;"""

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,orderLineNumber,priceEach,quantityOrdered,itemTotal,productCode,productName
0,1,107.34,36,3864.24,S24_2300,1962 Volkswagen Microbus
1,2,58.34,22,1283.48,S18_2432,1926 Ford Fire Engine
2,3,92.46,31,2866.26,S32_1268,1980’s GM Manhattan Express
3,4,119.67,42,5026.14,S10_4962,1962 LanciaA Delta 16V
4,5,98.07,36,3530.52,S18_4600,1940s Ford truck


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    FROM 
    orderdetails AS o,
    products AS p
    
    WHERE o.productCode = p.productCode AND orderNumber = 10103
    
    ORDER BY orderLineNumber
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

## Cálculo de totais e subtotais


#### Performance de venda por categoria de produtos.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productLine

ORDENE INVERSAMENTE POR
   lineTotal
```

In [40]:
query = """SELECT 
      p.productLine,
      count(p.productLine) as nItems,
      sum(o.priceEach * o.quantityOrdered) / count(p.productLine) AS averagePerItem,
      sum(o.priceEach * o.quantityOrdered) AS lineTotal
    
    FROM
    orderdetails AS o INNER JOIN products AS p

    WHERE
    o.productCode = p.productCode   

    GROUP BY
    p.productLine

    ORDER BY
    lineTotal DESC;"""

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,productLine,nItems,averagePerItem,lineTotal
0,Classic Cars,1010,3815.764842,3853922.49
1,Vintage Cars,657,2736.011613,1797559.63
2,Motorcycles,359,3123.749638,1121426.12
3,Trucks and Buses,308,3325.044058,1024113.57
4,Planes,336,2841.183155,954637.54


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM 
    orderdetails AS o, products AS p
    /*orderdetails AS o INNER JOIN products AS p*/
    
    GROUP BY p.productLine
    ORDER BY lineTotal
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Qual é o produto que mais vende ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productName,
    count(o.quantityOrdered) AS nItems,
#    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productCode

ORDENE INVERSAMENTE POR
   lineTotal
```

In [41]:
query = """SELECT 
      p.productName,
      count(o.quantityOrdered) AS nItems,
      sum(o.priceEach * o.quantityOrdered) AS lineTotal
    
    FROM
    orderdetails AS o INNER JOIN products AS p

    WHERE
    o.productCode = p.productCode   

    GROUP BY
    p.productCode

    ORDER BY
    lineTotal DESC;"""

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,productName,nItems,lineTotal
0,1992 Ferrari 360 Spider red,53,276839.98
1,2001 Ferrari Enzo,27,190755.86
2,1952 Alpine Renault 1300,28,190017.96
3,2003 Harley-Davidson Eagle Drag Bike,28,170686.00
4,1968 Ford Mustang,27,161531.48


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    p.productName,
    count(o.quantityOrdered) AS nItems,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM orderdetails o INNER JOIN products p
    
    WHERE o.productCode = p.productCode
    
    GROUP BY p.productCode
    
    ORDER BY lineTotal
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Qual é o cliente que mais compra ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,
    customers as c

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

MOSTRE AS COLUNAS
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    c.customerNumber

ORDENE INVERSAMENTE POR
   c.customerNumber
```

In [43]:
query = """SELECT 
      c.customerName,
      c.customerNumber,
      sum(od.priceEach * od.quantityOrdered) AS customerTotal,
      sum(od.quantityOrdered) AS nItems,
    
    FROM
    orderdetails AS od INNER JOIN orders AS o INNER JOIN customers AS c

    WHERE
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber   

    GROUP BY
    c.customerNumber

    ORDER BY
    nItems DESC;"""

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,customerName,customerNumber,customerTotal,nItems
0,Euro+ Shopping Channel,141,820689.54,9327
1,Mini Gifts Distributors Ltd.,124,591827.34,6366
2,"Australian Collectors, Co.",114,180585.07,1926
3,La Rochelle Gifts,119,158573.12,1832
4,"AV Stores, Co.",187,148410.09,1778


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal,
    sum(od.quantityOrdered) AS nItems
    
    FROM 
    orderdetails AS od INNER JOIN orders AS o INNER JOIN customers AS c

    WHERE od.orderNumber = o.orderNumber AND c.customerNumber = o.customerNumber

    GROUP BY c.customerNumber
    
    ORDER BY c.customerNumber
        
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult
-->

#### Vendas e valores por mês.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber

MOSTRE AS COLUNAS
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    month

ORDENE INVERSAMENTE POR
   month
```

In [45]:
query = """SELECT 
      substr(o.orderDate, 1, 7) AS month,
      sum(od.priceEach * od.quantityOrdered) AS monthTotal,
      sum(od.quantityOrdered) AS nItems
    
    FROM
    orderdetails AS od INNER JOIN orders AS o

    WHERE
    od.orderNumber = o.orderNumber 

    GROUP BY
    month

    ORDER BY
    month DESC;"""

queryResult = pd.read_sql_query(query, db)
queryResult.head()

,month,monthTotal,nItems
0,2005-05,441474.94,4759
1,2005-04,344820.62,3836
2,2005-03,359711.96,4207
3,2005-02,317192.17,3393
4,2005-01,307737.02,3395
